In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import choice
import pickle
import pandas as pd
from collections import defaultdict
import lyricsgenius

In [2]:
def get_keys():
    f = open('api_keys.txt', 'r')
    API_KEYS = f.readlines()
    
    for idx, key in enumerate(API_KEYS):
        API_KEYS[idx] = key.replace('\n', '')
        
    return API_KEYS

def get_random_query():
    '''
    Random choice from alphabet.

    Parameters
    ----------
    None:

    Returns
    ----------
    Wildcard for query : (string)
        Return random alphabet character and % (<char>%) from spotipy query.
    '''
    chars = 'abcdefghijklmnopqrstuvwxyz'
    return choice(list(chars)) + '%'

def get_all_album_ids(album_query):
    album_info = album_query['albums']['items']
    album_ids = []
    page_cnt = 50
    
    for page in range(page_cnt):
        for n in range(len(album_info)):
            album_ids.append(album_info[n]['id'])
        album_info = sp.next(album_query['albums'])['albums']['items']
    
    return album_ids

def album_builder(unique_albums):
    dfs_lst = []
    audio_feat_lst = ['song_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature']

    for album_id in unique_albums:
        album_dict = defaultdict(list)
        audio_feat = defaultdict(dict)
        tracks = sp.album_tracks(album_id)
        album_name = sp.album(album_id)['name']

        for track in pd.Series(tracks['items']):
            album_dict['album'].append(album_name)
            album_dict['album_id'].append(album_id)
            artist_lst = []
            for artist in track['artists']:
                artist_lst.append(artist['name'])

            album_dict['artist'].append(artist_lst)
            album_dict['artist_id'].append(track['artists'][0]['id'])
            album_dict['song'].append(track['name'])
            album_dict['song_id'].append(track['id'])
            audio_feat[track['id']] = sp.audio_features(track['id'])[0]

        feat_df = pd.DataFrame.from_dict(audio_feat, orient='index')
        feat_df.reset_index(inplace=True)
        feat_df.rename(columns={'index':'song_id'}, inplace=True)
        feat_df = feat_df[audio_feat_lst]
        album_df = pd.DataFrame.from_dict(album_dict)
        full_album_df = album_df.merge(feat_df, on='song_id')

        dfs_lst.append(full_album_df)
        del album_df
        del album_dict
        del audio_feat

    return pd.concat(dfs_lst)

def get_lyrics(song, artist):
    song = genius.search_song(song, artist)
    return song.lyrics

def get_full_lyrics(dfs_lst):
    for album in dfs_lst:
        lyrics_lst = []
        for idx, track in album.iterrows():
            try:
                lyrics_lst.append(get_lyrics(track.song, track.artist[0]))
            except:
                print('No lyrics for this song')
                song_error_df.append(track)
                continue

        try:
            album['lyrics'] = lyrics_lst
        except:
            err_cnt += 1
            print('Error applying lyrics')
            continue

    return dfs_lst

In [3]:
client_id, client_secret, client_access_token = get_keys()

In [4]:
#main
client_id, client_secret, client_access_token = get_keys()
q_type='album'
limit = 50
# album_ids = []

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                    client_secret=client_secret))
genius = lyricsgenius.Genius(client_access_token)

album_ids = []
chars = list('abcdefghijklmnopqrstuvwxyz')

album_query = sp.search(q='a%', limit=10, type=q_type)
album_ids.append(get_all_album_ids(album_query, sp))

unique_albums = pd.Series(album_ids[0]).unique()

final_albums_df = pd.concat(get_full_lyrics(album_builder(unique_albums, sp)))
# pickle.dump(final_albums_df, open('final_albums_df.pkl', 'wb'))

TypeError: get_all_album_ids() takes 1 positional argument but 2 were given

In [ ]:
final_albums_df

In [14]:
def get_keys():
    f = open('api_keys.txt', 'r')
    API_KEYS = f.readlines()

    for idx, key in enumerate(API_KEYS):
        API_KEYS[idx] = key.replace('\n', '')

    return API_KEYS

def get_random_query():
    '''
    Random choice from alphabet.

    Parameters
    ----------
    None:

    Returns
    ----------
    Wildcard for query : (string)
        Return random alphabet character and % (<char>%) from spotipy query.
    '''
    chars = 'abcdefghijklmnopqrstuvwxyz'
    return choice(list(chars)) + '%'

def get_all_album_ids(album_query, sp):
    album_info = album_query['albums']['items']
    album_ids = []
    page_cnt = 50

    for page in range(page_cnt):
        for n in range(len(album_info)):
            album_ids.append(album_info[n]['id'])
        album_info = sp.next(album_query['albums'])['albums']['items']

    return album_ids

def album_builder(unique_albums, sp):
    dfs_lst = []
    audio_feat_lst = ['song_id', 'danceability', 'energy', 'key', 'loudness',
    'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
    'valence', 'tempo', 'duration_ms', 'time_signature']

    for album_id in unique_albums:
        album_dict = defaultdict(list)
        audio_feat = defaultdict(dict)
        tracks = sp.album_tracks(album_id)
        album_name = sp.album(album_id)['name']

        for track in pd.Series(tracks['items']):
            album_dict['album'].append(album_name)
            album_dict['album_id'].append(album_id)
            artist_lst = []
            for artist in track['artists']:
                artist_lst.append(artist['name'])

            album_dict['artist'].append(artist_lst)
            album_dict['artist_id'].append(track['artists'][0]['id'])
            album_dict['song'].append(track['name'])
            album_dict['song_id'].append(track['id'])
            audio_feat[track['id']] = sp.audio_features(track['id'])[0]

        feat_df = pd.DataFrame.from_dict(audio_feat, orient='index')
        feat_df.reset_index(inplace=True)
        feat_df.rename(columns={'index':'song_id'}, inplace=True)
        feat_df = feat_df[audio_feat_lst]
        album_df = pd.DataFrame.from_dict(album_dict)
        full_album_df = album_df.merge(feat_df, on='song_id')

        dfs_lst.append(full_album_df)
        del album_df
        del album_dict
        del audio_feat

    return dfs_lst

def get_lyrics(song, artist):
    song = genius.search_song(song, artist)
    return song.lyrics

def get_full_lyrics(dfs_lst):
    for album in dfs_lst:
        lyrics_lst = []
        for idx, track in album.iterrows():
            try:
                lyrics_lst.append(get_lyrics(track.song, track.artist[0]))
            except:
                print('No lyrics for this song')
                continue

        try:
            album['lyrics'] = lyrics_lst
        except:
            print('Error applying lyrics')
            continue

    return dfs_lst


In [5]:
album_ids = pickle.load(open('unique_albums.pkl', 'rb'))

In [8]:
# album_ids = []
# chars = list('abcdefghijklmnopqrstuvwxyz')

# for letter in chars:
#     album_query = sp.search(q=letter + '%', limit=10, type=q_type)
#     album_ids.append(get_all_album_ids(album_query))

# len(album_ids)

In [14]:
def album_builder(unique_albums):
    dfs_lst = []
    audio_feat_lst = ['song_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature']

    for album_id in unique_albums:
        album_dict = defaultdict(list)
        audio_feat = defaultdict(dict)
        tracks = sp.album_tracks(album_id)
        album_name = sp.album(album_id)['name']

        for track in pd.Series(tracks['items']):
            album_dict['album'].append(album_name)
            album_dict['album_id'].append(album_id)
            artist_lst = []
            for artist in track['artists']:
                artist_lst.append(artist['name'])

            album_dict['artist'].append(artist_lst)
            album_dict['artist_id'].append(track['artists'][0]['id'])
            album_dict['song'].append(track['name'])
            album_dict['song_id'].append(track['id'])
            audio_feat[track['id']] = sp.audio_features(track['id'])[0]

        feat_df = pd.DataFrame.from_dict(audio_feat, orient='index')
        feat_df.reset_index(inplace=True)
        feat_df.rename(columns={'index':'song_id'}, inplace=True)
        feat_df = feat_df[audio_feat_lst]
        album_df = pd.DataFrame.from_dict(album_dict)
        full_album_df = album_df.merge(feat_df, on='song_id')

        dfs_lst.append(full_album_df)
        del album_df
        del album_dict
        del audio_feat

    return pd.concat(dfs_lst)

In [5]:
for album in dfs_lst:
    lyrics_lst = []
    for idx, track in album.iterrows():
        try:
            lyrics_lst.append(get_lyrics(track.song, track.artist[0]))
        except:
            print('No lyrics for this song')
            song_error_df.append(track)
            continue

    try:
        album['lyrics'] = lyrics_lst
    except:
        err_cnt += 1
        print('Error applying lyrics')
        continue

Searching for "Alone Again" by The Weeknd...
Done.
Searching for "Too Late" by The Weeknd...
Done.
Searching for "Hardest To Love" by The Weeknd...
Done.
Searching for "Scared To Live" by The Weeknd...
Done.
Searching for "Snowchild" by The Weeknd...
Done.
Searching for "Escape From LA" by The Weeknd...
Done.
Searching for "Heartless" by The Weeknd...
Done.
Searching for "Faith" by The Weeknd...
Done.
Searching for "Blinding Lights" by The Weeknd...
Done.
Searching for "In Your Eyes" by The Weeknd...
Done.
Searching for "Save Your Tears" by The Weeknd...
Done.
Searching for "Repeat After Me (Interlude)" by The Weeknd...
Done.
Searching for "After Hours" by The Weeknd...
Done.
Searching for "Until I Bleed Out" by The Weeknd...
Done.
Searching for "STARGAZING" by Travis Scott...
Done.
Searching for "CAROUSEL" by Travis Scott...
Done.
Searching for "SICKO MODE" by Travis Scott...
Done.
Searching for "R.I.P. SCREW" by Travis Scott...
Done.
Searching for "STOP TRYING TO BE GOD" by Travis Sc

Done.
Searching for "More Than Friends" by Quinn XCII...
Done.
Searching for "Mad At Me" by Quinn XCII...
Done.
Searching for "Two 10s" by Quinn XCII...
Done.
Searching for "Meeting Strangers" by Quinn XCII...
Done.
Searching for "Everything I Need" by Quinn XCII...
Done.
Searching for "Do I Wanna Know?" by Arctic Monkeys...
Done.
Searching for "R U Mine?" by Arctic Monkeys...
Done.
Searching for "One For The Road" by Arctic Monkeys...
Done.
Searching for "Arabella" by Arctic Monkeys...
Done.
Searching for "I Want It All" by Arctic Monkeys...
Done.
Searching for "No. 1 Party Anthem" by Arctic Monkeys...
Done.
Searching for "Mad Sounds" by Arctic Monkeys...
Done.
Searching for "Fireside" by Arctic Monkeys...
Done.
Searching for "Why'd You Only Call Me When You're High?" by Arctic Monkeys...
Done.
Searching for "Snap Out Of It" by Arctic Monkeys...
Done.
Searching for "Knee Socks" by Arctic Monkeys...
Done.
Searching for "I Wanna Be Yours" by Arctic Monkeys...
Done.
Searching for "Come T

Done.
Searching for "Max B (feat. Joe Fox)" by A$AP Rocky...
Done.
Searching for "Pharsyde (feat. Joe Fox)" by A$AP Rocky...
Done.
Searching for "Wavybone (feat. Juicy J & UGK)" by A$AP Rocky...
Done.
Searching for "West Side Highway (feat. James Fauntleroy)" by A$AP Rocky...
Done.
Searching for "Better Things" by A$AP Rocky...
Done.
Searching for "M'$ (feat. Lil Wayne)" by A$AP Rocky...
Done.
Searching for "Dreams (Interlude)" by A$AP Rocky...
Done.
Searching for "Everyday (feat. Rod Stewart, Miguel & Mark Ronson)" by A$AP Rocky...
Done.
Searching for "Back Home (feat. Mos Def, Acyde & Yams)" by A$AP Rocky...
Done.
Searching for "Introduction" by Panic! At The Disco...
Done.
Searching for "The Only Difference Between Martyrdom and Suicide Is Press Coverage" by Panic! At The Disco...
Done.
Searching for "London Beckoned Songs About Money Written by Machines" by Panic! At The Disco...
Done.
Searching for "Nails for Breakfast, Tacks for Snacks" by Panic! At The Disco...
Done.
Searching f

Done.
Searching for "WHY AM I STILL IN LA (feat. Shlohmo & D33J)" by Joji...
Done.
Searching for "NO FUN" by Joji...
Done.
Searching for "COME THRU" by Joji...
Done.
Searching for "R.I.P. (feat. Trippie Redd)" by Joji...
Done.
Searching for "XNXX" by Joji...
Done.
Searching for "I'LL SEE YOU IN 40" by Joji...
Done.
Searching for "Message (feat. TyFontaine)" by Internet Money...
Done.
Searching for "Really Redd (feat. Trippie Redd, Lil Keed, Young Nudy)" by Internet Money...
Done.
Searching for "Lost Me (feat. Lil Mosey, iann dior & Lil Skies)" by Internet Money...
Done.
Searching for "Right Now (feat. Cochise & TyFontaine)" by Internet Money...
Done.
Searching for "Familiar (feat. TheHxliday)" by Internet Money...
Done.
Searching for "JLO (feat. Lil Tecca)" by Internet Money...
Done.
Searching for "Thrusting (feat. Swae Lee & Future)" by Internet Money...
Done.
Searching for "Speak (feat. The Kid Laroi)" by Internet Money...
Done.
Searching for "Blastoff (feat. Juice Wrld & Trippie Red

Done.
Searching for "I Am" by James Arthur...
Done.
Searching for "Train Wreck" by James Arthur...
Done.
Searching for "Safe Inside" by James Arthur...
Done.
Searching for "Sober" by James Arthur...
Done.
Searching for "Phoenix" by James Arthur...
Done.
Searching for "Let Me Love the Lonely" by James Arthur...
Done.
Searching for "Sermon (feat. Shotty Horroh)" by James Arthur...
Done.
Searching for "Remember Who I Was" by James Arthur...
Done.
Searching for "Finally" by James Arthur...
Done.
Searching for "The Truth" by James Arthur...
Done.
Searching for "Skeletons" by James Arthur...
Done.
Searching for "If Only" by James Arthur...
Done.
Searching for "Coming Home for Summer" by James Arthur...
Done.
Searching for "Hells Bells" by AC/DC...
Done.
Searching for "Shoot to Thrill" by AC/DC...
Done.
Searching for "What Do You Do for Money Honey" by AC/DC...
Done.
Searching for "Givin the Dog a Bone" by AC/DC...
Done.
Searching for "Let Me Put My Love Into You" by AC/DC...
Done.
Searching 

...

Done.
Searching for "Invisible Klip" by Kid Trunks...
Done.
Searching for "Bowser" by XXXTENTACION...
Done.
Searching for "One" by Ed Sheeran...
Done.
Searching for "I'm a Mess" by Ed Sheeran...
Done.
Searching for "Sing" by Ed Sheeran...
Done.
Searching for "Don't" by Ed Sheeran...
Done.
Searching for "Nina" by Ed Sheeran...
Done.
Searching for "Photograph" by Ed Sheeran...
Done.
Searching for "Bloodstream" by Ed Sheeran...
Done.
Searching for "Tenerife Sea" by Ed Sheeran...
Done.
Searching for "Runaway" by Ed Sheeran...
Done.
Searching for "The Man" by Ed Sheeran...
Done.
Searching for "Thinking out Loud" by Ed Sheeran...
Done.
Searching for "Afire Love" by Ed Sheeran...
Done.
Searching for "Take It Back" by Ed Sheeran...
Done.
Searching for "Shirtsleeves" by Ed Sheeran...
Done.
Searching for "Even My Dad Does Sometimes" by Ed Sheeran...
Done.
Searching for "I See Fire" by Ed Sheeran...
Done.
Searching for "All of the Stars" by Ed Sheeran...
Done.
Searching for "English Rose" by Ed S

Done.
Error applying lyrics
Searching for "Love Never Felt So Good" by Michael Jackson...
Done.
Searching for "Chicago" by Michael Jackson...
Done.
Searching for "Loving You" by Michael Jackson...
Done.
Searching for "A Place With No Name" by Michael Jackson...
Done.
Searching for "Slave to the Rhythm" by Michael Jackson...
Done.
Searching for "Do You Know Where Your Children Are" by Michael Jackson...
Done.
Searching for "Blue Gangsta" by Michael Jackson...
Done.
Searching for "Xscape" by Michael Jackson...
Done.
Searching for "Love Never Felt So Good (Original Version)" by Michael Jackson...
Done.
Searching for "Chicago (Original Version)" by Michael Jackson...
Done.
Searching for "Loving You (Original Version)" by Michael Jackson...
Done.
Searching for "A Place With No Name (Original Version)" by Michael Jackson...
Done.
Searching for "Slave to the Rhythm (Original Version)" by Michael Jackson...
Done.
Searching for "Do You Know Where Your Children Are (Original Version)" by Michael

Done.
Searching for "Oh My Sweet Carolina - Live" by Zac Brown Band...
Done.
Searching for "Nothing" by Zac Brown Band...
Done.
Searching for "It's You" by Ali Gatie...
Done.
Searching for "Moonlight" by Ali Gatie...
Done.
Searching for "Used to You" by Ali Gatie...
Done.
Searching for "Say to You" by Ali Gatie...
Done.
Searching for "Love You Like That" by Ali Gatie...
Done.
Searching for "Sunshine" by Ali Gatie...
Done.
Searching for "How Things Used to Be" by Ali Gatie...
Done.
Searching for "Holding On" by Ali Gatie...
Done.
Searching for "Lost My Lover" by Ali Gatie...
Done.
Searching for "Catch Me Outside" by Ski Mask The Slump God...
Done.
Searching for "Take A Step Back" by Ski Mask The Slump God...
Done.
Searching for "WTF!?" by Ski Mask The Slump God...
Done.
Searching for "Rambo" by Ski Mask The Slump God...
Done.
Searching for "JustLikeMyPiss" by Ski Mask The Slump God...
Done.
Searching for "Bird Is The Word" by Ski Mask The Slump God...
Done.
Searching for "BabyWipe" by S

Done.
Searching for "It Get Deeper" by 42 Dugg...
Done.
Searching for "Been Turnt" by 42 Dugg...
Done.
Searching for "Hard Times" by 42 Dugg...
Done.
Searching for "Turnt Bitch" by 42 Dugg...
Done.
Searching for "Bout 40" by 42 Dugg...
Done.
Searching for "One of One (feat. Babyface Ray)" by 42 Dugg...
Done.
Searching for "Ride With Me" by 42 Dugg...
Done.
Searching for "World On My Shoulder" by 42 Dugg...
Done.
Searching for "Palm Angels In The Sky" by 42 Dugg...
Done.
Searching for "Bounce Back" by 42 Dugg...
Done.
Searching for "1997" by Key Glock...
Done.
Searching for "Dough" by Key Glock...
Done.
Searching for "Word on the Streets" by Key Glock...
Done.
Searching for "Ooh" by Key Glock...
Done.
Searching for "What Goes Around Comes Around" by Key Glock...
Done.
Searching for "Crash" by Key Glock...
Done.
Searching for "Look At They Face" by Key Glock...
Done.
Searching for "I'm Just Sayin" by Key Glock...
Done.
Searching for "Biig Boyy!" by Key Glock...
Done.
Searching for "Flyes

Done.
Searching for "Man In A Suitcase" by The Police...
Done.
Searching for "Shadows In The Rain" by The Police...
Done.
Searching for "The Other Way Of Stopping" by The Police...
Done.
Searching for "Try Everything" by Shakira...
Done.
Searching for "Stage Fright" by Michael Giacchino...
Done.
Searching for "Grey's uh-Mad at Me" by Michael Giacchino...
Done.
Searching for "Ticket to Write" by Michael Giacchino...
Done.
Searching for "Foxy Fakeout" by Michael Giacchino...
Done.
Searching for "Jumbo Pop Hustle" by Michael Giacchino...
Done.
Searching for "Walk and Stalk" by Michael Giacchino...
Done.
Searching for "Not a Real Cop" by Michael Giacchino...
Done.
Searching for "Hopps Goes (After) the Weasel" by Michael Giacchino...
Done.
Searching for "The Naturalist" by Michael Giacchino...
Done.
Searching for "Work Slowly and Carry a Big Shtick" by Michael Giacchino...
Done.
Searching for "Mr. Big" by Michael Giacchino...
Done.
Searching for "Case of the Manchas" by Michael Giacchino...

## saving lyric json streamlining

In [163]:
for idx, track in dfs_lst[0].iterrows():
    name = ''.join(track.artist[0].lower())
    track_name = track
    print(f'lyrics_{track.song.lower()}_{name}')

lyrics_alone again_the weeknd
lyrics_too late_the weeknd
lyrics_hardest to love_the weeknd
lyrics_scared to live_the weeknd
lyrics_snowchild_the weeknd
lyrics_escape from la_the weeknd
lyrics_heartless_the weeknd
lyrics_faith_the weeknd
lyrics_blinding lights_the weeknd
lyrics_in your eyes_the weeknd
lyrics_save your tears_the weeknd
lyrics_repeat after me (interlude)_the weeknd
lyrics_after hours_the weeknd
lyrics_until i bleed out_the weeknd


In [165]:
from string import punctuation

In [178]:
my_punct = punctuation + ' '
punctuation, my_punct

('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~', '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [183]:
for track in dfs_lst[0].song:
    print(track.translate(str.maketrans('', '', my_punct)).lower())

aloneagain
toolate
hardesttolove
scaredtolive
snowchild
escapefromla
heartless
faith
blindinglights
inyoureyes
saveyourtears
repeataftermeinterlude
afterhours
untilibleedout


In [4]:
ec2_df = pickle.load(open('final_albums_df.pkl', 'rb'))

In [5]:
ec2_df.reset_index()

,index,album,album_id,artist,artist_id,song,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,After Hours,4yP0hdKOZPNshxUOjY0cZj,[The Weeknd],1Xyo4u8uXC1ZmMpatF05PJ,Alone Again,6b5P51m8xx2XA6U7sdNZ5E,0.218,0.652,4,-7.953,0,0.0641,0.0330,0.111000,0.1720,0.0599,65.375,250053,4
1,1,After Hours,4yP0hdKOZPNshxUOjY0cZj,[The Weeknd],1Xyo4u8uXC1ZmMpatF05PJ,Too Late,2K9Ovn1o2bTGqbsABGC6m3,0.638,0.863,4,-5.221,0,0.1320,0.0520,0.000055,0.6850,0.2260,120.077,239973,4
2,2,After Hours,4yP0hdKOZPNshxUOjY0cZj,[The Weeknd],1Xyo4u8uXC1ZmMpatF05PJ,Hardest To Love,6kWxIqQDsKFYCJGbU4AjCX,0.467,0.608,10,-7.847,1,0.0707,0.0909,0.000088,0.1250,0.1080,80.957,211400,4
3,3,After Hours,4yP0hdKOZPNshxUOjY0cZj,[The Weeknd],1Xyo4u8uXC1ZmMpatF05PJ,Scared To Live,4BGZF4oLbTL0pWm7C18pbv,0.504,0.498,0,-8.240,1,0.0499,0.1310,0.000000,0.0929,0.1960,87.221,191307,4
4,4,After Hours,4yP0hdKOZPNshxUOjY0cZj,[The Weeknd],1Xyo4u8uXC1ZmMpatF05PJ,Snowchild,3WlbeuhfRSqU7ylK2Ui5U7,0.550,0.611,0,-7.475,1,0.1370,0.1620,0.000069,0.1210,0.2430,148.062,247187,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,6,Artist,2OQEAqShAl6SodrGhmYZ4Z,[A Boogie Wit da Hoodie],31W5EY0aAly4Qieq6OFu6I,1hunnit,54zRmxMXR0OE0qkMuIm9Ve,0.700,0.481,3,-10.296,0,0.2730,0.3490,0.000000,0.1120,0.3070,88.047,247954,4
325,7,Artist,2OQEAqShAl6SodrGhmYZ4Z,[A Boogie Wit da Hoodie],31W5EY0aAly4Qieq6OFu6I,Money over Everything,3QEF7zOUrhLz65F7oUftU6,0.646,0.795,1,-10.298,1,0.3240,0.3770,0.000000,0.1210,0.6210,133.624,211139,4
326,8,Artist,2OQEAqShAl6SodrGhmYZ4Z,[A Boogie Wit da Hoodie],31W5EY0aAly4Qieq6OFu6I,Trap House,2pFwdWMfP4pu3FUhCCWkDT,0.779,0.714,6,-5.203,0,0.2060,0.3090,0.000000,0.0765,0.3980,135.974,186253,4
327,9,Artist,2OQEAqShAl6SodrGhmYZ4Z,[A Boogie Wit da Hoodie],31W5EY0aAly4Qieq6OFu6I,Fall in Love,1sXJ0TZDdYBIV4mlzQFi4j,0.842,0.727,0,-6.241,1,0.0675,0.2610,0.000000,0.1310,0.7210,107.994,214857,4


In [6]:
ec2_df.album.unique()

array(['After Hours', 'ASTROWORLD', 'American Teen', 'AI YoungBoy 2',
       'A Love Letter To You 4', 'Artist 2.0', 'ANTI (Deluxe)',
       'A Love Letter To You 3', 'A Letter To My Younger Self', 'AM',
       'Abbey Road (Remastered)', 'Appetite For Destruction',
       'All Eyez On Me', 'AT.LONG.LAST.A$AP', 'Artist 2.0 (Deluxe)',
       "A Fever You Can't Sweat Out", 'ADHD',
       'American Beauty/American Psycho', 'A Love Letter To You',
       'Artist'], dtype=object)

In [8]:
albums = ec2_df.groupby('album').mean()

In [5]:
client_id, client_secret, client_access_token = get_keys()
q_type='album'
limit = 50
# album_ids = []

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                    client_secret=client_secret))
genius = lyricsgenius.Genius(client_access_token)

In [10]:
album_ids = pickle.load(open('../../data/dump/pickle/dfs_w_lyrics.pkl', 'rb'))

In [13]:
album_df = pd.concat(album_ids)

In [14]:
album_df

,album,album_id,artist,artist_id,song,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics
0,After Hours,4yP0hdKOZPNshxUOjY0cZj,[The Weeknd],1Xyo4u8uXC1ZmMpatF05PJ,Alone Again,6b5P51m8xx2XA6U7sdNZ5E,0.218,0.65200,4,-7.953,0,0.0641,0.03300,0.111000,0.1720,0.0599,65.375,250053,4,[Verse]\nTake off my disguise\nI'm living some...
1,After Hours,4yP0hdKOZPNshxUOjY0cZj,[The Weeknd],1Xyo4u8uXC1ZmMpatF05PJ,Too Late,2K9Ovn1o2bTGqbsABGC6m3,0.638,0.86300,4,-5.221,0,0.1320,0.05200,0.000055,0.6850,0.2260,120.077,239973,4,"[Intro]\nNo-no, no-no, no-no-no\n\n[Verse 1]\n..."
2,After Hours,4yP0hdKOZPNshxUOjY0cZj,[The Weeknd],1Xyo4u8uXC1ZmMpatF05PJ,Hardest To Love,6kWxIqQDsKFYCJGbU4AjCX,0.467,0.60800,10,-7.847,1,0.0707,0.09090,0.000088,0.1250,0.1080,80.957,211400,4,"[Intro]\nNo, yeah\nOh, yeah\nOh, yeah\nYeah, u..."
3,After Hours,4yP0hdKOZPNshxUOjY0cZj,[The Weeknd],1Xyo4u8uXC1ZmMpatF05PJ,Scared To Live,4BGZF4oLbTL0pWm7C18pbv,0.504,0.49800,0,-8.240,1,0.0499,0.13100,0.000000,0.0929,0.1960,87.221,191307,4,"[Verse 1]\nWhen I saw the signs, I shoulda let..."
4,After Hours,4yP0hdKOZPNshxUOjY0cZj,[The Weeknd],1Xyo4u8uXC1ZmMpatF05PJ,Snowchild,3WlbeuhfRSqU7ylK2Ui5U7,0.550,0.61100,0,-7.475,1,0.1370,0.16200,0.000069,0.1210,0.2430,148.062,247187,4,[Verse 1]\nI used to pray when I was sixteen\n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,zero_one,54wAlSP6p1gVxZmlzAfpiJ,[The Living Tombstone],0I6yADrmeyvw66g7yjkqIF,Fly Home,4QKu8B7OZfELPkQlwtwe3X,0.660,0.74800,10,-4.568,1,0.0305,0.00500,0.000000,0.2420,0.5610,104.020,207253,4,"[Verse 1]\nPack your bag, hop in a car\nAnothe..."
9,zero_one,54wAlSP6p1gVxZmlzAfpiJ,[The Living Tombstone],0I6yADrmeyvw66g7yjkqIF,Chosen,1hjYmCKINchQvPyzZRlgmu,0.310,0.80200,7,-5.322,1,0.1060,0.00633,0.000000,0.0929,0.0636,159.750,201813,4,[Intro]\nWill I be myself again?\nWhere does i...
0,Zip-A-Dee-Doo-Dah,40DkJu7Ucdoj25THYEybUl,[Penelope Lang],384BZ30OneQUlMKtjHTVIf,Zip-A-Dee-Doo-Dah,4sP8BKTHCC1CEdp4mOVkdZ,0.457,0.01340,10,-25.149,1,0.0452,0.99500,0.925000,0.1230,0.4050,113.567,160500,3,NaN
1,Zip-A-Dee-Doo-Dah,40DkJu7Ucdoj25THYEybUl,[Penelope Lang],384BZ30OneQUlMKtjHTVIf,Over The River And Through The Woods,7sVzX9PWTOJEX6QDWPm6U2,0.495,0.02070,1,-26.898,1,0.0535,0.99600,0.963000,0.0925,0.5180,108.070,135875,4,NaN


In [20]:
for id_ in album_df.artist_id.unique(): # flow for mongo genre df
    print(sp.artist(id_)['genres'])

['canadian contemporary r&b', 'canadian pop', 'pop']
['rap']
['alternative r&b', 'pop']
['baton rouge rap', 'trap']
['melodic rap', 'rap', 'trap']
['melodic rap', 'pop rap', 'rap']
['barbadian pop', 'dance pop', 'pop', 'post-teen pop', 'r&b', 'urban contemporary']
['indie pop rap', 'pop', 'pop rap']
['garage rock', 'modern rock', 'permanent wave', 'rock', 'sheffield indie']
['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']
['glam metal', 'hard rock', 'rock']
['g funk', 'gangster rap', 'hip hop', 'rap', 'west coast rap']
['east coast hip hop', 'hip hop', 'pop rap', 'rap', 'southern hip hop', 'trap']
['baroque pop', 'emo', 'modern rock', 'pop punk']
['boston hip hop', 'hip hop', 'pop rap', 'rap']
['emo', 'modern rock', 'pop punk']
['north carolina hip hop', 'rap']
['dfw rap', 'melodic rap', 'rap']
['alternative r&b', 'hip hop', 'lgbtq+ hip hop', 'neo soul', 'pop']
['alternative r&b', 'pop', 'viral pop']
['pop rap', 'rap', 'trap']
['atl hip hop',

['dance pop', 'hip pop', 'neo soul', 'pop', 'pop r&b', 'pop rap', 'r&b', 'urban contemporary']
['bedroom pop', 'indie pop', 'pop']
['detroit hip hop', 'hip hop', 'rap']
[]
['conscious hip hop', 'hip hop', 'north carolina hip hop', 'rap', 'southern hip hop']
['conscious hip hop', 'east coast hip hop', 'gangster rap', 'hardcore hip hop', 'hip hop', 'queens hip hop', 'rap']
['dance pop', 'pop', 'post-teen pop']
['dance pop', 'pop']
['canadian contemporary r&b', 'canadian pop', 'dance pop', 'electropop', 'pop', 'post-teen pop']
['hip hop', 'rap', 'underground hip hop']
['alternative r&b', 'indie r&b', 'pop', 'pop r&b', 'post-teen pop', 'r&b', 'urban contemporary']
['contemporary country', 'country', 'country road']
['contemporary country', 'pop']
['gangster rap', 'harlem hip hop', 'hip hop', 'nyc rap', 'pop rap', 'rap', 'southern hip hop', 'trap']
['latin', 'reggaeton', 'trap latino']
['latin', 'latin hip hop', 'reggaeton', 'tropical']
['latin', 'reggaeton', 'trap latino']
['melodic rap', 

['dance pop', 'pop', 'pop rap', 'r&b', 'urban contemporary']
['album rock', 'classic rock', 'country rock', 'rock', 'roots rock', 'southern rock', 'swamp rock']
['east coast hip hop', 'hip hop', 'pop rap', 'rap']
['acoustic pop', 'neo mellow', 'pop', 'pop rock', 'viral pop']
['dark trap', 'emo rap', 'underground hip hop', 'vapor trap']
['dark trap', 'underground hip hop', 'vapor trap']
['asian american hip hop', 'dark trap', 'underground hip hop', 'vapor trap']
[]
['dark trap', 'miami hip hop', 'underground hip hop', 'vapor trap']
[]
['dance pop', 'edm', 'liquid funk', 'pop', 'tropical house', 'uk dance', 'uk funky']
[]
['dark trap']
['dark trap', 'emo rap', 'underground hip hop']
[]
['downtempo', 'dream pop', 'indietronica']
['indie pop', 'modern alternative rock', 'modern rock']
['jam band', 'neo mellow', 'pop rock']
['pop', 'r&b', 'soul']
['neo mellow', 'pop', 'pop rock', 'singer-songwriter']
['bachata', 'latin', 'tropical']
[]
['latin', 'latin hip hop', 'reggaeton', 'tropical']
['p